<a href="https://colab.research.google.com/github/rrwiren/ilmanlaatu-ennuste-helsinki/blob/main/PIPELINE_v0.5.4_ESIK%C3%84SITTELY%2C_EDA_PIIRTEET_BASELINE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
============================================================
 SOLU 1: Alustus, Datan Lataus ja Esikäsittely (v0.5.4 pohja)
============================================================
"""
# === 1. Alustus ja Kirjastojen Tuonti ===
print("="*60); print(" Solu 1: Alustus ja Kirjastojen Tuonti"); print("="*60)
import pandas as pd; import numpy as np; import requests; import io; import os
import matplotlib.pyplot as plt; import seaborn as sns;
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib
try: import pyarrow; print(f"'pyarrow' löytyy (versio: {pyarrow.__version__}).")
except ImportError: print("VAROITUS: 'pyarrow' puuttuu."); pyarrow = None
# Tulostetaan muiden versiot, jos ne on asennettu
try: import prophet as prophet_module; from prophet import Prophet; print(f"--- Prophet-versio: {prophet_module.__version__} ---")
except ImportError: print("INFO: 'prophet'-kirjastoa ei löydy.")
try: import xgboost as xgb; print(f"--- XGBoost-versio: {xgb.__version__} ---")
except ImportError: print("INFO: 'xgboost'-kirjastoa ei löydy.")
try: import shap; print(f"--- SHAP-versio: {shap.__version__} ---")
except ImportError: print("INFO: 'shap'-kirjastoa ei löydy.")

plt.style.use('ggplot')

# --- Konfiguraatio ---
DATA_URLS = {
    "weather_21_23": "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kaisaniemi_%2015.4.2021%20-%2016.4.2023_ee091df1-3113-44e8-9191-631ce34ce08b.csv",
    "weather_23_25": "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kaisaniemi_%2015.4.2023%20-%2015.4.2025_020c3655-5fd6-442b-8569-f296327d4630.csv",
    "aq_20_25":      "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/raw/Helsinki%20Kallio%202_%2015.4.2020%20-%2015.4.2025_6f14506f-9460-49ac-b5bc-609aaa60d82b.csv"
}
print(f"\nKäytetään {len(DATA_URLS)} datatiedostoa:")
for key, url in DATA_URLS.items(): print(f" - {key}: ...{url[-100:]}")
START_DATE = pd.Timestamp('2021-04-15 00:00:00'); print(f"\nData suodatetaan alkamaan: {START_DATE.date()}")
TARGET_VARIABLE = 'Ozone'; TEST_DATA_RATIO = 0.2; RANDOM_STATE = 42
SAVE_FINAL_DATA = True; SAVE_BASELINE_MODEL = True # Näitä käytetään myöhemmissä soluissa
PROCESSED_DATA_FOLDER = "data/processed"; MODEL_FOLDER = "models"
PROCESSED_DATA_FILENAME_FE = f"Helsinki_Data_2021-2025_With_Features_Pipeline_v0.5.4.parquet"
BASELINE_MODEL_FILENAME = f"baseline_lr_2021-2025_v0.5.4.joblib"
PROCESSED_DATA_PATH_FE = os.path.join(PROCESSED_DATA_FOLDER, PROCESSED_DATA_FILENAME_FE)
BASELINE_MODEL_SAVE_PATH = os.path.join(MODEL_FOLDER, BASELINE_MODEL_FILENAME)


# === Apufunktiot (Lataus & Esikäsittely) ===
def _load_single_csv_from_url(url, key_name):
    print(f"  Ladataan {key_name}...");
    try: response = requests.get(url); response.raise_for_status(); df = pd.read_csv(io.StringIO(response.text), sep=','); print(f"    -> Ladattu ({df.shape[0]} riviä)."); return df
    except Exception as e: print(f"    -> VIRHE ({key_name}): {e}"); return None

def _preprocess_single_fmi_df(df, data_key_name):
    """Esikäsittelee YHDEN FMI DataFrame:n (korjattu try/except)."""
    if df is None: return None
    print(f"  Esikäsitellään (aika, numerot): {data_key_name}")
    original_cols = df.columns.tolist(); date_cols = ['Vuosi', 'Kuukausi', 'Päivä']; time_col = 'Aika [Paikallinen aika]'
    if not all(col in original_cols for col in date_cols + [time_col]): missing_cols = [col for col in date_cols + [time_col] if col not in original_cols]; print(f"    -> VIRHE: Aikaleimasarakkeita puuttuu ({data_key_name}). Puuttuvat: {missing_cols}"); return None
    df_processed = None
    try:
        dt_str = df['Vuosi'].astype(str)+'-'+df['Kuukausi'].astype(str).str.zfill(2)+'-'+df['Päivä'].astype(str).str.zfill(2)+' '+df[time_col].astype(str)
        df['Timestamp'] = pd.to_datetime(dt_str); cols_to_drop = date_cols + [time_col] + ['Havaintoasema']
        df_processed = df.drop(columns=cols_to_drop, errors='ignore').set_index('Timestamp'); # print(f"    -> Aikaleima & indeksi OK ({data_key_name}).")
    except Exception as e: print(f"    -> VIRHE aikaleiman luonnissa/indeksoinnissa ({data_key_name}): {e}."); return None
    if df_processed is not None:
        # print(f"    -> Muunnetaan numeeriseksi ({data_key_name})...")
        for col in df_processed.columns:
            if not pd.api.types.is_numeric_dtype(df_processed[col]): df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
        df_processed = df_processed.dropna(axis=1, how='all'); print(f"    -> Esikäsittely valmis ({data_key_name}). Jäljellä: {df_processed.columns.tolist()}"); return df_processed
    else: return None # Jos try epäonnistui

# === Pipeline Vaihe 1: Datan Lataus ===
def lataa_raakadata(url_dict):
    """Lataa raakadatatiedostot määritellyistä URL-osoitteista."""
    print("\n" + "="*60); print(" Pipeline Vaihe 1: Datan Lataus"); print("="*60)
    raw_dataframes = {}; success = True
    for key, url in url_dict.items(): df_raw = _load_single_csv_from_url(url, key); raw_dataframes[key] = df_raw; success &= (df_raw is not None)
    if not success: print("\nVAROITUS: Datan lataus epäonnistui osittain."); return None
    print("\nKaikki pyydetyt datatiedostot ladattu."); return raw_dataframes

# === Pipeline Vaihe 2: Datan Esikäsittely, Yhdistäminen, Suodatus & Siivous ===
def esikasittele_ja_yhdistä_data(raw_dfs_dict, start_date):
    """Esikäsittelee, yhdistää säädatan, suodattaa päivämäärän ja siivoaa."""
    print("\n" + "="*60); print(" Pipeline Vaihe 2: Esikäsittely, Yhdistäminen & Suodatus"); print("="*60)
    required_keys = ['weather_21_23', 'weather_23_25', 'aq_20_25'];
    if not isinstance(raw_dfs_dict, dict) or not all(key in raw_dfs_dict for key in required_keys): print(f"VIRHE: Odotettuja avaimia ({required_keys}) puuttuu."); return None
    print("Esikäsitellään yksittäiset tiedostot..."); df_w1 = _preprocess_single_fmi_df(raw_dfs_dict.get('weather_21_23'), 'weather_21_23'); df_w2 = _preprocess_single_fmi_df(raw_dfs_dict.get('weather_23_25'), 'weather_23_25'); df_aq_raw = _preprocess_single_fmi_df(raw_dfs_dict.get('aq_20_25'), 'aq_20_25')
    if df_w1 is None or df_w2 is None or df_aq_raw is None: print("VIRHE: Esikäsittely epäonnistui."); return None
    print("\nUudelleennimetään sarakkeita...")
    rename_map_weather_base = {'Ilman lämpötila keskiarvo [°C]': 'T','Tuulen suunta keskiarvo [°]': 'WD','Keskituulen nopeus keskiarvo [m/s]': 'WS','Näkyvyys keskiarvo [m]': 'Vis','Ilmanpaine merenpinnan tasolla keskiarvo [hPa]': 'P_Sea','Pilvisyys [1/8]': 'Cloud','Ylin lämpötila [°C]': 'T_Max','Alin lämpötila [°C]': 'T_Min','Ilmanpaineen keskiarvo [hPa]': 'P_W2'}
    rename_map_w1 = {k: v+"_W1" for k, v in rename_map_weather_base.items() if k in df_w1.columns}; df_w1 = df_w1.rename(columns=rename_map_w1); print(f"  -> weather1: {df_w1.columns.tolist()}")
    rename_map_w2 = {k: v+"_W2" for k, v in rename_map_weather_base.items() if k in df_w2.columns}; df_w2 = df_w2.rename(columns=rename_map_w2); print(f"  -> weather2: {df_w2.columns.tolist()}")
    aq_map = {'Otsoni [µg/m3]': 'Ozone', 'Hengitettävät hiukkaset <10 µm [µg/m3]': 'PM10','Pienhiukkaset <2.5 µm [µg/m3]': 'PM25','Ilmanlaatuindeksi [1/5]': 'AQI', 'Typpidioksidi [µg/m3]': 'NO2', 'Typpimonoksidi [µg/m3]': 'NO', 'Hiilimonoksidi [µg/m3]': 'CO', 'Rikkidioksidi [µg/m3]': 'SO2', 'Musta hiili [µg/m3]': 'BC'}
    aq_cols = [k for k in aq_map.keys() if k in df_aq_raw.columns]; v_map_aq = {k: v for k, v in aq_map.items() if k in aq_cols}; df_aq_raw = df_aq_raw[aq_cols].rename(columns=v_map_aq); print(f"  -> aq: {df_aq_raw.columns.tolist()}")
    if TARGET_VARIABLE not in df_aq_raw.columns: print(f"VIRHE: Kohde '{TARGET_VARIABLE}' puuttuu AQ datasta!"); return None
    print("\nYhdistetään säädatat (concat)..."); df_w = pd.concat([df_w1, df_w2], axis=0).sort_index()
    if not df_w.index.is_unique: print(f"-> Käsitellään {df_w.index.duplicated().sum()} säädatan duplikaattia..."); df_w = df_w.groupby(level=0).mean(); print("-> Duplikaatit käsitelty.")
    print(f"  -> Yhdistetty säädata: {df_w.shape}")
    print(f"\nSuodatetaan data alkamaan {start_date.date()}..."); df_w_f = df_w[df_w.index >= start_date].copy(); df_aq_f = df_aq_raw[df_aq_raw.index >= start_date].copy(); print(f"  -> Säädata: {df_w_f.shape} ({df_w_f.index.min()} - {df_w_f.index.max()})"); print(f"  -> AQ-data: {df_aq_f.shape} ({df_aq_f.index.min()} - {df_aq_f.index.max()})")
    if df_w_f.empty or df_aq_f.empty: print("VIRHE: Suodatuksen jälkeen data tyhjä!"); return None
    print("\nYhdistetään suodatettu sää- ja ilmanlaatudata..."); df_m = pd.merge(df_w_f, df_aq_f, left_index=True, right_index=True, how='outer'); print(f"  -> Yhdistetty (outer). Muoto: {df_m.shape}")
    print("\nSuoritetaan lopullinen siivous..."); df_m = df_m.sort_index()
    if not df_m.index.is_unique: print(f"-> Käsitellään {df_m.index.duplicated().sum()} merge-duplikaattia..."); df_m = df_m.groupby(level=0).mean(); print("-> Duplikaatit käsitelty.")
    else: print("-> Ei duplikaatteja.")
    if not df_m.empty: print("-> Varmistetaan tuntifrekvenssi..."); min_ts, max_ts = df_m.index.min(), df_m.index.max(); print(f"   Aikaväli: {min_ts} - {max_ts}"); df_m = df_m.reindex(pd.date_range(start=min_ts, end=max_ts, freq='h')); print(f"-> Uusi muoto: {df_m.shape}")
    else: print("VIRHE: Data tyhjä."); return None
    print("-> Täytetään NaN..."); nan_b = df_m.isnull().sum().sum(); df_m = df_m.ffill().bfill()
    if df_m.isnull().sum().sum()==0: print(f"-> NaN täytetty (oli {nan_b}).")
    else: print(f"-> VAROITUS: {df_m.isnull().sum().sum()} NaN jäi!"); return None
    print("\nSiivotaan/yhdistetään päällekkäiset sääsarakkeet..."); df_c = df_m.copy()
    dup_pairs = {'Temperature': ('T_W1', 'T_W2'), 'WindSpeed': ('WS_W1', 'WS_W2'), 'WindDirection': ('WD_W1', 'WD_W2'), 'Pressure': ('P_Sea_W1', 'P_W2'), 'Visibility': ('Vis_W1', 'Vis_W2')} # Päivitetty parit
    cols_drop = []
    # **TÄMÄN SILMUKAN SISENNYS TARKISTETTU**
    for new_name, (c1, c2) in dup_pairs.items():
        if c1 in df_c.columns and c2 in df_c.columns:
            print(f"  Yhdistetään: {c1} & {c2} -> {new_name}")
            if new_name == 'WindDirection': # Käytetään vektorikeskiarvoa
                 r1=np.deg2rad(df_c[c1]); r2=np.deg2rad(df_c[c2]); x=(np.cos(r1)+np.cos(r2))/2.; y=(np.sin(r1)+np.sin(r2))/2.
                 df_c[new_name]=(np.rad2deg(np.arctan2(y, x))+360)%360
            else: # Muille normaali keskiarvo
                 df_c[new_name]=df_c[[c1,c2]].mean(axis=1)
            cols_drop.extend([c1, c2])
        elif c1 in df_c.columns: print(f"  -> Käytetään {c1} nimellä {new_name} ({c2} puuttui)."); df_c = df_c.rename(columns={c1: new_name})
        elif c2 in df_c.columns: print(f"  -> Käytetään {c2} nimellä {new_name} ({c1} puuttui)."); df_c = df_c.rename(columns={c2: new_name})
        else: print(f"  -> HUOM: Paria {c1}/{c2} ei löytynyt.")
    cols_drop_f = [c for c in list(set(cols_drop)) if c in df_c.columns];
    if cols_drop_f: df_c=df_c.drop(columns=cols_drop_f); print(f"  -> Poistettu _W1/_W2: {cols_drop_f}")
    # Poistetaan lopuksi varmuuden vuoksi kaikki jäljelle jääneet _Wx päätteiset sarakkeet
    cols_to_remove_single = [col for col in df_c.columns if '_W1' in col or '_W2' in col]
    if cols_to_remove_single: df_c = df_c.drop(columns=cols_to_remove_single); print(f"  -> Poistettu parittomat _W1/_W2: {cols_to_remove_single}")
    print("  -> Päällekkäisten sarakkeiden käsittely valmis.")
    print("\nKorjataan negatiiviset pitoisuudet..."); conc_cols = ['Ozone','PM10','PM25','NO2','NO','SO2','BC','CO', 'AQI']
    cols_chk = [c for c in conc_cols if c in df_c.columns]; neg_c = (df_c[cols_chk] < 0).sum()
    if (neg_c > 0).any(): print("  Määrät:\n", neg_c[neg_c > 0].to_string())
    for c in cols_chk:
        if neg_c.get(c, 0) > 0: df_c[c]=df_c[c].clip(lower=0)
    print("  -> Korjattu.") if (neg_c > 0).any() else print("  -> Ei negatiivisia.")
    print("\nEsikäsittely, yhdistäminen ja siivous valmis."); print("Lopulliset sarakkeet:", df_c.columns.tolist()); return df_c

# === SUORITUS: Ladataan ja esikäsitellään data ===
# Tämä osa ajetaan nyt solun lopussa, jotta df_valmis on seuraavan solun käytettävissä
if __name__ == "__main__": # Tämä tarkistus toimii myös notebook-solussa
    raaka_datat_dict = lataa_raakadata(DATA_URLS)
    if raaka_datat_dict:
        df_valmis = esikasittele_ja_yhdistä_data(raaka_datat_dict, START_DATE)
        if df_valmis is not None:
             print("\n" + "*"*30)
             print(" Datan esikäsittely valmis! ")
             print(f" Lopullisen DataFramen ('df_valmis') muoto: {df_valmis.shape}")
             print(" Ensimmäiset 5 riviä:")
             print(df_valmis.head())
             print("*"*30)
        else:
             print("\nDatan esikäsittely epäonnistui.")
    else:
         print("\nDatan lataus epäonnistui.")

In [ ]:
# -*- coding: utf-8 -*-
"""
============================================================
 SOLU 2: EDA ja Piirteiden Muokkaus (v0.5.4 pohja)
============================================================
 OLETUS: df_valmis -DataFrame on luotu edellisessä solussa.
"""
# Tuodaan kirjastot uudelleen tähän soluun varmuuden vuoksi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os # Tarvitaan tallennuspolkujen luontiin myöhemmin

# === Tarkistetaan df_valmis olemassaolo ===
if 'df_valmis' not in locals() or df_valmis is None:
    print("VIRHE: Muuttujaa 'df_valmis' ei löydy tai se on tyhjä.")
    print("Aja edellinen solu (Solu 1: Alustus, Datan Lataus ja Esikäsittely) ensin onnistuneesti.")
    # Voit halutessasi pysäyttää suorituksen tässä
    # raise NameError("'df_valmis' ei ole määritelty. Aja edellinen solu.")
else:
    print(f"Käytetään 'df_valmis' DataFramea (muoto: {df_valmis.shape}).")

    # === Määritellään tarvittavat globaalit muuttujat (kopioitu konfiguraatiosta) ===
    # Nämä tarvitaan funktioiden sisällä
    TARGET_VARIABLE = 'Ozone'
    RANDOM_STATE = 42 # Käytetään esim. SHAP-otoskoossa, jos sitä käytettäisiin

    # === Pipeline Vaihe 3: Datan Tutkiminen (EDA) ===
    def tutki_dataa(df):
        """Suorittaa EDA:n valmiiksi siivotulle DataFramelle."""
        print("\n" + "="*60); print(" Pipeline Vaihe 3: Datan Tutkiminen (EDA)"); print("="*60)
        if df is None or df.empty: print("VIRHE: Ei dataa tutkittavaksi."); return
        # Tehdään kopio varmuuden vuoksi, vaikka tätä ei pitäisi muokata
        df_eda = df.copy()
        print("\n--- 3a: Kohdemuuttujan ('Ozone') Analyysi ---")
        if TARGET_VARIABLE not in df_eda.columns: print(f"VIRHE: Kohde '{TARGET_VARIABLE}' puuttuu!"); return
        print(f"  Tilastot '{TARGET_VARIABLE}':"); print(df_eda[TARGET_VARIABLE].describe().to_string())
        print(f"  Visualisoidaan '{TARGET_VARIABLE}' aikasarja..."); plt.figure(figsize=(18, 5)); df_eda[TARGET_VARIABLE].plot(alpha=0.8, title=f'{TARGET_VARIABLE} ajan funktiona ({df_eda.index.min().date()} - {df_eda.index.max().date()})'); plt.ylabel(f'{TARGET_VARIABLE} (µg/m³)'); plt.xlabel('Aika'); plt.grid(True); plt.tight_layout(); plt.show()
        print(f"  Visualisoidaan '{TARGET_VARIABLE}' jakauma..."); plt.figure(figsize=(10, 5)); sns.histplot(df_eda[TARGET_VARIABLE], kde=True, bins=50); plt.title(f'{TARGET_VARIABLE} jakauma'); plt.xlabel(f'{TARGET_VARIABLE} (µg/m³)'); plt.ylabel('Frekvenssi / Tiheys'); plt.grid(True, axis='y'); plt.tight_layout(); plt.show()
        print("\n--- 3b: Korrelaatioanalyysi ---"); w_cols = [TARGET_VARIABLE, 'Temperature', 'WindSpeed', 'WindDirection', 'Pressure', 'Visibility', 'T_Max', 'T_Min', 'Cloud']; w_cols = [c for c in w_cols if c in df_eda.columns]; print(f"  Korrelaatioanalyysiin valitut sarakkeet: {w_cols}")
        if len(w_cols) > 1: corr_m = df_eda[w_cols].corr(); plt.figure(figsize=(9, 7)); sns.heatmap(corr_m, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5); plt.title(f'Korrelaatiot: {TARGET_VARIABLE} vs. Sää'); plt.show(); print(f"\n  {TARGET_VARIABLE} korrelaatiot (laskeva):"); print(corr_m[TARGET_VARIABLE].sort_values(ascending=False).drop(TARGET_VARIABLE).to_string())
        else: print("  -> Ei tarpeeksi sarakkeita korrelaatioon.")
        print("\n--- 3c: Hajontakaaviot ---"); scatter_f = ['Temperature', 'WindSpeed', 'Visibility', 'Pressure', 'WindDirection']; scatter_f = [c for c in scatter_f if c in df_eda.columns]; print(f"  Piirretään: {TARGET_VARIABLE} vs {scatter_f}")
        # --- TÄMÄN IF/ELSE -LOHKON SISENNYS ON KRIITTINEN ---
        if len(scatter_f) > 0:
            # IF:n sisällä oleva koodi
            n_f=len(scatter_f); n_r=(n_f+1)//2; plt.figure(figsize=(14, 5*n_r));
            for i, f in enumerate(scatter_f):
                 plt.subplot(n_r, 2, i+1); sns.scatterplot(data=df_eda, x=f, y=TARGET_VARIABLE, alpha=0.1, s=5); plt.title(f'{TARGET_VARIABLE} vs. {f}'); plt.xlabel(f); plt.ylabel(f'{TARGET_VARIABLE} (µg/m³)'); plt.grid(True)
            plt.tight_layout();
            plt.show() # Tämä rivi IF-lohkon lopussa
        # ELSE samalla tasolla kuin IF
        else:
            # Tämä rivi sisennetty ELSE:n alle
            print("  -> Ei piirteitä hajontakaavioita varten.")
        # --- IF/ELSE PÄÄTTYY ---
        print("\n--- 3d: Ajallinen Analyysi ---");
        # --- TÄMÄN IF/ELSE -LOHKON SISENNYS ON KRIITTINEN ---
        if 'Temperature' in df_eda.columns: # IF alkaa
            # IF:n sisällä oleva koodi
            print("  Kuukausivaihtelu..."); m_avg = df_eda.groupby(df_eda.index.month)[[TARGET_VARIABLE, 'Temperature']].mean(); m_avg.index.name = 'Kuukausi'; fig, ax1 = plt.subplots(figsize=(12, 5)); ax2 = ax1.twinx(); m_avg[TARGET_VARIABLE].plot(ax=ax1, c='r', marker='o', label=TARGET_VARIABLE); m_avg['Temperature'].plot(ax=ax2, c='b', marker='s', ls='--', label='Lämpötila'); ax1.set(ylabel=f'Keskim. {TARGET_VARIABLE} (µg/m³)', xlabel='Kuukausi', title='Keskim. kuukausivaihtelu', xticks=m_avg.index); ax1.tick_params(axis='y', labelcolor='r'); ax2.set_ylabel('Keskim. Lämpötila (°C)', color='b'); ax2.tick_params(axis='y', labelcolor='b'); ax1.grid(True); fig.legend(loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax1.transAxes); fig.tight_layout(); plt.show()
            print("  Tuntivaihtelu..."); h_avg = df_eda.groupby(df_eda.index.hour)[[TARGET_VARIABLE, 'Temperature']].mean(); h_avg.index.name = 'Tunti'; fig, ax1 = plt.subplots(figsize=(12, 5)); ax2 = ax1.twinx(); h_avg[TARGET_VARIABLE].plot(ax=ax1, c='r', marker='o', label=TARGET_VARIABLE); h_avg['Temperature'].plot(ax=ax2, c='b', marker='s', ls='--', label='Lämpötila'); ax1.set(ylabel=f'Keskim. {TARGET_VARIABLE} (µg/m³)', xlabel='Tunti', title='Keskim. tuntivaihtelu', xticks=h_avg.index[::2]); ax1.tick_params(axis='y', labelcolor='r'); ax2.set_ylabel('Keskim. Lämpötila (°C)', color='b'); ax2.tick_params(axis='y', labelcolor='b'); ax1.grid(True); fig.legend(loc="upper right", bbox_to_anchor=(1,1), bbox_transform=ax1.transAxes); fig.tight_layout(); plt.show()
        # ELSE samalla tasolla kuin IF
        else:
             # Tämä rivi sisennetty ELSE:n alle
             print("  -> Lämpötila puuttuu, ei voida piirtää ajallisia vertailuja.")
        # --- IF/ELSE PÄÄTTYY ---
        print("\nEDA-vaihe suoritettu.")


    # === Pipeline Vaihe 4: Piirteiden Muokkaus ===
    def muokkaa_piirteet(df):
        """Lisää dataan aikaan perustuvia piirteitä."""
        print("\n" + "="*60); print(" Pipeline Vaihe 4: Piirteiden Muokkaus"); print("="*60)
        if df is None or df.empty or not isinstance(df.index, pd.DatetimeIndex): print("VIRHE: Vaatii DataFrame:n."); return None, None
        df_feat = df.copy(); print("Lisätään aikapiirteitä...")
        idx = df_feat.index; df_feat['hour'] = idx.hour; df_feat['dayofweek'] = idx.dayofweek; df_feat['dayofyear'] = idx.dayofyear
        df_feat['month'] = idx.month; df_feat['year'] = idx.year; df_feat['weekofyear'] = idx.isocalendar().week.astype(int)
        print("  -> Lineaariset lisätty: hour, dayofweek, dayofyear, month, year, weekofyear")
        df_feat['hour_sin']=np.sin(2*np.pi*df_feat['hour']/24.0); df_feat['hour_cos']=np.cos(2*np.pi*df_feat['hour']/24.0)
        df_feat['dayofweek_sin']=np.sin(2*np.pi*df_feat['dayofweek']/7.0); df_feat['dayofweek_cos']=np.cos(2*np.pi*df_feat['dayofweek']/7.0)
        df_feat['month_sin']=np.sin(2*np.pi*df_feat['month']/12.0); df_feat['month_cos']=np.cos(2*np.pi*df_feat['month']/12.0)
        print("  -> Sykliset lisätty: hour_sin/cos, dayofweek_sin/cos, month_sin/cos")
        time_feature_names = ['hour','dayofweek','dayofyear','month','year','weekofyear','hour_sin','hour_cos','dayofweek_sin','dayofweek_cos','month_sin','month_cos']
        print(f"  -> Palautetaan lisätyt piirteet nimilistana.")
        print("\nPiirteiden muokkaus valmis."); return df_feat, time_feature_names


    # === SUORITUS: Ajetaan EDA ja Piirteiden muokkaus ===
    # Oletetaan, että df_valmis on luotu Solussa 1
    try:
        # Suoritetaan EDA (tulostaa ja plottaa)
        tutki_dataa(df_valmis)

        # Suoritetaan piirteiden muokkaus ja otetaan tulokset talteen
        # seuraavaa solua (Solu 3) varten
        df_piirteilla, aikapiirteiden_nimet = muokkaa_piirteet(df_valmis)

        if df_piirteilla is not None:
             print("\n" + "*"*30)
             print(" Solu 2 valmis! ")
             print(f" Lopullisen DataFramen ('df_piirteilla') muoto: {df_piirteilla.shape}")
             print(" Data sisältää nyt alkuperäiset siivotut piirteet sekä uudet aikapiirteet.")
             print(" Voit nyt ajaa Solu 3:n (Mallinnus ja tallennus).")
             print("*"*30)
             print("\nMuutama viimeinen rivi datasta piirteineen:")
             print(df_piirteilla.tail(3)) # Tulostetaan lopuksi muutama rivi
        else:
             print("\nPiirteiden muokkaus epäonnistui. Tarkista aiemmat virheet.")

    except NameError:
         print("\nVIRHE: Muuttujaa 'df_valmis' ei löydy. Aja Solu 1 ensin onnistuneesti.")
    except Exception as e:
         print(f"\nTapahtui odottamaton virhe Solussa 2: {e}")
         import traceback
         traceback.print_exc()

In [ ]:
# -*- coding: utf-8 -*-
"""
==========================================================================================
 SOLU 3: Datan Jako, Baseline-mallin Koulutus, Evaluointi & Tallennus (v0.5.4 pohja)
==========================================================================================
 OLETUS: df_piirteilla -DataFrame ja aikapiirteiden_nimet -lista on luotu edellisessä solussa.
"""

# === Tuodaan tarvittavat kirjastot uudelleen varmuuden vuoksi ===
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
import joblib

# === Tarkistetaan edellisen solun muuttujien olemassaolo ===
if 'df_piirteilla' not in locals() or df_piirteilla is None or 'aikapiirteiden_nimet' not in locals() or aikapiirteiden_nimet is None:
    print("VIRHE: Tarvittavia muuttujia ('df_piirteilla', 'aikapiirteiden_nimet') ei löydy tai ne ovat tyhjiä.")
    print("Aja edellinen solu (Solu 2: EDA ja Piirteiden Muokkaus) ensin onnistuneesti.")
    # raise NameError("Tarvittavat muuttujat edellisestä solusta puuttuvat.") # Voit poistaa kommentin
else:
    print(f"Käytetään 'df_piirteilla' DataFramea (muoto: {df_piirteilla.shape}) ja {len(aikapiirteiden_nimet)} aikapiirrettä.")

    # === Määritellään tarvittavat globaalit muuttujat (kopioitu konfiguraatiosta) ===
    TARGET_VARIABLE = 'Ozone'
    TEST_DATA_RATIO = 0.2
    SAVE_FINAL_DATA = True
    SAVE_BASELINE_MODEL = True
    PROCESSED_DATA_PATH_FE = "data/processed/Helsinki_Data_2021-2025_With_Features_Pipeline_v0.5.4.parquet" # Tiedostonimi tallennusta varten
    BASELINE_MODEL_SAVE_PATH = "models/baseline_lr_2021-2025_v0.5.4.joblib" # Tiedostonimi tallennusta varten


    # === Pipeline Vaiheet 5-9 Funktioina ===
    # Määritellään funktiot tässä solussa, jotta se on itsenäisempi

    def jaa_data(df, target_sarake, test_osuus):
        """Jakaa datan kronologisesti opetus- ja testijoukkoihin."""
        print("\n" + "="*60); print(" Pipeline Vaihe 5: Datan Jako"); print("="*60)
        if df is None or df.empty: print("VIRHE: Ei dataa."); return None, None, None, None
        if target_sarake not in df.columns: print(f"VIRHE: Kohde '{target_sarake}' puuttuu."); return None, None, None, None
        n=len(df); split_index=int(n*(1-test_osuus));
        if split_index<=0 or split_index>=n: print(f"VIRHE: Testikoko {test_osuus} virheellinen."); return None,None,None,None
        df_train=df.iloc[:split_index]; df_test=df.iloc[split_index:]
        print(f"Data jaettu: Opetus {df_train.shape[0]} ({df_train.index.min()} - {df_train.index.max()})")
        print(f"            Testi  {df_test.shape[0]} ({df_test.index.min()} - {df_test.index.max()})")
        feature_columns = df.columns.drop(target_sarake); X_train = df_train[feature_columns]; y_train = df_train[target_sarake]; X_test = df_test[feature_columns]; y_test = df_test[target_sarake]
        print("Opetus/testijoukot luotu (X,y)."); return X_train, y_train, X_test, y_test

    def kouluta_perusmalli(X_train, y_train, aikapiirteet):
        """Kouluttaa Lineaarisen Regression baseline-mallin käyttäen vain aikapiirteitä."""
        print("\n" + "="*60); print(" Pipeline Vaihe 6: Baseline-mallin Koulutus"); print("="*60)
        if X_train is None or y_train is None or not aikapiirteet: print("VIRHE: Virheelliset syötteet."); return None
        missing = [p for p in aikapiirteet if p not in X_train.columns];
        if missing: print(f"VIRHE: Aikapiirteitä puuttuu X_trainista: {missing}."); return None
        print(f"Koulutetaan LR piirteillä: {aikapiirteet}")
        X_train_time = X_train[aikapiirteet]; model = LinearRegression()
        try: model.fit(X_train_time, y_train); print("Baseline-malli koulutettu."); return model
        except Exception as e: print(f"VIRHE koulutuksessa: {e}"); return None

    def evaluoi_malli(model, X, y, datasetti_nimi, aikapiirteet=None):
        """Evaluoi koulutetun mallin suorituskyvyn annetulla datajoukolla."""
        print("\n" + "="*60); print(f" Pipeline Vaihe 7: Mallin Evaluointi ({datasetti_nimi})"); print("="*60)
        if model is None or X is None or y is None: print("VIRHE: Virheelliset syötteet."); return None
        X_eval = X; features_used = X.columns.tolist()
        if aikapiirteet: # Jos evaluoidaan baseline-mallia joka käyttää vain aikapiirteitä
            missing = [p for p in aikapiirteet if p not in X.columns];
            if missing: print(f"VIRHE: Aikapiirteitä puuttuu X:stä: {missing}."); return None
            X_eval = X[aikapiirteet]; features_used = aikapiirteet
            print(f"Käytetään vain piirteitä: {features_used}")
        try:
            y_pred = model.predict(X_eval); mae = mean_absolute_error(y, y_pred); rmse = np.sqrt(mean_squared_error(y, y_pred)); r2 = r2_score(y, y_pred)
            print(f"Suorituskykymetriikat ({datasetti_nimi}): MAE={mae:.3f}, RMSE={rmse:.3f}, R²={r2:.3f}")
            plt.figure(figsize=(18, 5)); plt.plot(y.index, y, label='Todelliset', alpha=0.8); plt.plot(y.index, y_pred, label='Ennusteet', alpha=0.8, linestyle='--')
            plt.title(f'Mallin Ennusteet vs. Todelliset ({datasetti_nimi})'); plt.xlabel('Aika'); plt.ylabel(TARGET_VARIABLE); plt.legend(); plt.grid(True); plt.tight_layout(); plt.show()
            return {'MAE': mae, 'RMSE': rmse, 'R2': r2}
        except Exception as e: print(f"VIRHE evaluoinnissa ({datasetti_nimi}): {e}"); return None

    # Korjattu tallennusfunktio
    def tallenna_lopullinen_data(df, polku):
        """Tallentaa annetun DataFramen Parquet-tiedostoon."""
        print("\n" + "="*60); print(" Pipeline Vaihe 8: Datan Tallennus (Parquet)"); print("="*60)
        if df is None or df.empty: print("VIRHE: Ei dataa."); return False
        global pyarrow # Käytetään globaalia pyarrow-tarkistusta
        if not pyarrow: print("VAROITUS: 'pyarrow' puuttuu."); return False
        print("'pyarrow' löytyy.")
        try:
            output_folder=os.path.dirname(polku);
            if output_folder: os.makedirs(output_folder, exist_ok=True)
            df.to_parquet(polku, index=True);
            print(f"\nLopullinen data tallennettu: {polku}");
            return True
        except Exception as e:
            print(f"\nVIRHE tallennuksessa {polku}: {e}");
            return False

    # Korjattu tallennusfunktio
    def tallenna_malli(model, polku):
        """Tallentaa koulutetun mallin levylle käyttäen joblibia."""
        print("\n" + "="*60); print(" Pipeline Vaihe 9: Mallin Tallennus"); print("="*60)
        if model is None: print("VIRHE: Ei mallia."); return False
        try:
            output_folder = os.path.dirname(polku);
            if output_folder: os.makedirs(output_folder, exist_ok=True)
            joblib.dump(model, polku);
            print(f"Malli tallennettu: {polku}");
            return True
        except Exception as e:
            print(f"VIRHE mallin tallennuksessa {polku}: {e}");
            return False

    # === Ajetaan vaiheet 5-9 ===
    print("\n" + "="*80); print(" AJETAAN MALLINNUS JA TALLENNUS (Solu 3)"); print("="*80)
    X_train, y_train, X_test, y_test = None, None, None, None
    baseline_model = None; baseline_results = None
    tallennettu_ok_data = False; tallennettu_ok_malli = False

    try:
        # Vaihe 5: Jaa data (käyttäen df_piirteilla Solusta 2)
        X_train, y_train, X_test, y_test = jaa_data(df_piirteilla, TARGET_VARIABLE, test_osuus=TEST_DATA_RATIO)
        if X_train is None: raise ValueError("5. Datan jako epäonnistui.")

        # Vaihe 6: Kouluta baseline-malli (käyttäen aikapiirteiden_nimet Solusta 2)
        if aikapiirteiden_nimet: baseline_model = kouluta_perusmalli(X_train, y_train, aikapiirteiden_nimet)
        else: raise ValueError("6a. Aikapiirteiden nimiä ei saatu koulutukseen.")
        if baseline_model is None: raise ValueError("6b. Baseline-mallin koulutus epäonnistui.")

        # Vaihe 7: Evaluoi baseline-malli
        if aikapiirteiden_nimet: baseline_results = evaluoi_malli(baseline_model, X_test, y_test, "Testisetti (Baseline v0.5.4)", aikapiirteet=aikapiirteiden_nimet)
        else: raise ValueError("7a. Aikapiirteiden nimiä ei saatu evaluointiin.")
        if baseline_results is None: raise ValueError("7b. Baseline-mallin evaluointi epäonnistui.")

        # Tulostetaan baseline-tulokset selkeästi
        if baseline_results:
            print("\n" + "-"*40); print(" BASELINE MALLIN (v0.5.4) LOPULLISET METRIIKAT "); print("-"*40)
            print(f"  MAE:  {baseline_results['MAE']:.3f}")
            print(f"  RMSE: {baseline_results['RMSE']:.3f}")
            print(f"  R²:   {baseline_results['R2']:.3f}")
            print("-"*40)

        # Vaihe 8: Tallenna data piirteineen (df_piirteilla Solusta 2)
        if SAVE_FINAL_DATA and df_piirteilla is not None:
             # Tulostetaan vielä yhteenveto tallennettavasta datasta
             print("\n" + "-"*40); print("LOPULLISEN DATAFRAMEN YHTEENVETO"); print("-"*40)
             print("\nInfo:"); df_piirteilla.info()
             tallennettu_ok_data = tallenna_lopullinen_data(df_piirteilla, PROCESSED_DATA_PATH_FE)
             if tallennettu_ok_data: print(" -> Datan tallennus onnistui.")
             else: print(" -> Datan tallennus EI ONNISTUNUT.") # Selkeämpi virheilmoitus

        # Vaihe 9: Tallenna baseline-malli
        if SAVE_BASELINE_MODEL and baseline_model is not None:
             tallennettu_ok_malli = tallenna_malli(baseline_model, BASELINE_MODEL_SAVE_PATH)
             if tallennettu_ok_malli: print(" -> Baseline-mallin tallennus onnistui.")
             else: print(" -> Baseline-mallin tallennus EI ONNISTUNUT.") # Selkeämpi virheilmoitus

    except NameError as ne:
        print(f"\nVIRHE: Tarvittavaa muuttujaa ei löytynyt: {ne}")
        print("Varmista, että Solu 1 ja Solu 2 on ajettu onnistuneesti ennen tätä solua.")
    except Exception as e:
        print("\n" + "!"*80); print(f" KRIITTINEN VIRHE SUORITUKSESSA (Solu 3): {e}");
        import traceback; traceback.print_exc(); print("!"*80)
    finally:
        print("\n" + "="*80); print(f" MALLINNUSPIPELINE (Solu 3) SUORITETTU LOPPUUN"); print("="*80)